In [ ]:
verify_domains1([1,2,5,7],[3,4,8,10],10)

True

In [ ]:
e=[3,4,8,10]
b=[1,2,5,7]
ne=np.array(e)
nb=np.array(b)
nn=ne-nb+1
nn

array([3, 3, 4, 4])

In [ ]:
verify_domains2([1,2,5,7],[3,4,8,10],10,True)


[[1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]]
[[  62187.   79647.   79647.   62186.       0.       0.       0.       0.
        0.       0.]
 [  79647.  102010.  102010.   79647.       0.       0.       0.       0.
        0.       0.]
 [  79647.  102010.  102010.   79647.       0.       0.       0.       0.
        0.       0.]
 [  62186.   79647.   79647.   62187.       0.       0.       0.       0.
        0.       0.]
 [      0.       0.       0.       0.  861184.  861184. 1217536. 1217536.
   860672.  860672.]
 [      0.       0.       0.       0.  861184.  861184. 1217536. 1217536.
   860672.  860672.]
 [      0.       0.       0.       0. 

False

These Schwarz algorithms require 1) that the subdomains cover all dimensions in the space and 2) that there aren't disjoint sets of subdomains that overlap e.g. $b=[1,2,5,7], e=[3,4,8,10]$. We need to determine if a set of subdomains has full overlap. We do this by making a specific linear transformation from the subdomains and observing how information is transferred in the space.

We create a matrix $M$ of size $nxn$. For each subdomain (size $d$), we make a matrix of size $dxd$. We then place each matrix on the diagonal of $M$. We then raise $M$ to the nth power and multiply it by a unit vector like $e_{n-1}$. We are treating $M$ like a lienar transformation. If we raise it the the nth power, then $e_{n-1}$ should contribute to every dimension in $1..n$.


This use of the concept of information transfer should work for subdomains that are sets rather than ranges of vectors. With disjoint sets of subdomains, each set is represented by a subspace in the matrix. The column space of each subspace is itself, so no information can be transfered among subspaces.

<del> each no information could get from the positions one disjoint set to the other.

In [ ]:
import numpy as np
def verify_domains1(b,e,n):
  vec=[0]*n
  p=len(b)
  for i in range(p):
    if b[i]>e[i]:
      return False
  for i in range(p):
    for j in range(b[i],e[i]+1):
      vec[j-1]=1
  for s in vec:
    if s==0:
      return False
  return True
def verify_domains2(b,e,n,verbose=False):
  p=len(b)
  M=np.zeros((n,n))
  vec=np.zeros((n,1))
  vec[-1]=1
  for i in range(p):
    for j in range(b[i],e[i]+1):
      for l in range(b[i],e[i]+1):
        M[j-1,l-1]=1
  if verbose:
    print(M)
    print(vec)
  M=np.linalg.matrix_power(M,n)#probably more efficient to associate from the right side
  vec=M@vec
  if verbose:
    print(M)
    print(vec)
  for s in vec:
    if s==0:
      return False
  return True

def Multiplicative_Schwarz(K, n=10, p=4, b=[1,2,4,7], e=[3,4,8,10], iprint=0):
  #n=10
  #p=4
  
  # n=[3,3,5,4]
  # b=[1,2,4,7]
  # e=[3,4,8,10]
  # n=e-b+1
  assert(n<=100)
  assert(p<=40 and p <=n)
  assert(K<=1000)
  assert(verify_domains1(b,e,n))
  assert(verify_domains2(b,e,n))
  
  M,x,y,xstore=Linear_system(n)
  if iprint >= 2:
    print('xstore \n')
    print(xstore)
    if iprint >=4:
        print(M)
        print(y)
  nn,nb,ne,index= fix_domains(n,p,b,e,iprint)
  print(fix_domains(n,p,b,e,iprint))
  R=Rmatricies(n,p,nn,index,iprint)
  print(R) #verified the first 6 of 10
  A=Amatricies(M,R,n,p,nn,iprint)
  B=Bmatricies(A,R,n,p,nn,iprint)
  C=Cmatricies(M,B,n,p,iprint) 
  D=Dmatricies_multiplicative(C,n,p,iprint)
  f=fvector(B,y,n,p,iprint)
  vstore= Diteration_multiplicative(D,f,n,p,K,iprint)
  # print(vstore)
  S= Smatrix_multiplicative(D,n,p,iprint); #just creates it. D seems to be a substitute for C. DD is Ci.
  print(S)
  
  T= Tmatrix_multiplicative(D,n,p,iprint);
  #g= gvector_multiplicative(f,n,p,iprint);

def Additive_Schwarz(K,alpha,restricted,iprint):
  n=10
  p=4
  M,x,y,xstore=Linear_system(n)
  if iprint >= 2:
    print('xstore \n')
    print(xstore)
    if iprint >=4:
        print(M)
        print(y)
  nn,nb,ne,index= fix_domains(n,p,iprint);
  print(fix_domains(n,p,iprint))
  R=Rmatricies(n,p,nn,index,iprint)
  print(R)
  A=Amatricies(M,R,n,p,nn,iprint) #
  B=Bmatricies(A,R,n,p,nn,iprint)
  C=Cmatricies(M,B,n,p,iprint)
  f=fvector(B,y,n,p,iprint)
  vstore=Citeration_additive(C,f,alpha,n,p,K,iprint)
  D= Dmatrix_additive(C,alpha,n,p,iprint)
  g= gvector_additive(f,alpha,n,p,iprint)
  wstore= Diteration_additive(D,g,n,p,K,iprint)
  if iprint >= 2:
    print('wstore \n')
    #print(wstore(n*(K-1)+1:n*K)')
  U= Umatrix_additive(D,n,K,iprint)
  print(U)
  h= hvector_additive(g,n,K,iprint);

  # fprintf('Solve Uz= h \n')
  zstore= Uhsolve_additive(U,h,n,K,iprint)
 
  #% Compare solutions from all three methods
  #solution_compare(xstore,vstore(n*(K-1)+1:n*K),vstore,wstore,zstore,iprint)

  #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  #%                                                                              %
  #%     Add discretization error and compute error estimate                      %
  #%                                                                              %
  #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


  # K iterations computing subdomain contributions independently
  #------------------------------------------------------------

  psi= np.ones((n,1))
  print('\nPerforming C iteration with error \n')
  #vglobal_store,vaglobal_store,rglobal_store= Citeration_additive_approx(C,f,alpha,n,p,K,iprint)


## Additive Utilities

---I will still move these to different files.

### For C iteration

In [ ]:
def Citeration_additive(C,f,alpha,n,p,K,iprint):
  vstore= np.zeros((n*K,1))
  v= np.zeros((n,1))

  for k in range(0,K):
    dv= np.zeros((n,p))

    # Calculate and store subdomain contributions (independently)
    for i in range(0,p):
        CC= C[i,0:n,0:n]
        ff= f[i,0:n]
        print("top",np.shape(ff))
        ff=ff[np.newaxis]
        print(np.shape(ff))
        print(np.shape(CC))
        dv[0:n,i]= ff-CC @ v[0:n,0]
        if iprint >= 4:
            print('Outer iteration %3i, Inner iteration %3i \n',k,i)
            print(dv.T)
    
    # Perform update by summing weighted subdomain contributions
    for i in range(p):
        v[0:n,0]= v[0:n,0] + alpha*dv[0:n,i]
    
    if iprint >= 2:
        print('Outer iteration %3i \n',k)
        print(v.T)
        
    
    # Store iterates in a 1D vector
    vstore[k*n:(k+1)*(n)]= v[range(n)]#0:n,0]
    #vstore[k*n*p + i*n : k*n*p+(i+1)*n]=v[range(n)] #from mult
  return vstore



### For D iteration

In [ ]:
def hvector_additive(g,n,K,iprint):
  #Construct h vector for additive Schwarz

  h= np.zeros((K*n,1))
  for i in range(K):
    h[i*n:(i+1)*n,0:1]= g  ##should apply the 0:1 to other functions with same problem
  if iprint >= 4:
    print('h vector \n')
    print(h)
  return h


In [ ]:
def Diteration_additive(D,g,n,p,K,iprint):
  wstore= np.zeros((n*K,1))
  w= np.zeros((n,1))

  # Perform K iterations
  for k in range(K):
    # print(D)
    print(g)
    # print(w)
    w= D@w+g;
    wstore[(k)*n:(k+1)*n]= w[range(n)]
    if iprint >= 2:
        print('Outer iteration %3i \n',k)
        print(w.T)
  return wstore


In [ ]:
def Dmatrix_additive(C,alpha,n,p,iprint):
  D= np.zeros((n,n))
  for i in range(p):
    CC= C[i,0:n,0:n] #it complained when I tried to assign these values to CC[0:n,0:n]
    D= D+CC

  D= np.identity(n)-alpha*D;
  if iprint >= 4:
    print('Print DD matrix \n')
    print(D)
  return D


In [ ]:
def gvector_additive(f,alpha,n,p,iprint):

# Construct g vector for additive Schwarz


  g= np.zeros((n,1))
  for i in range(p):
    ff= f[i]
    ff=ff.reshape((10,1))
    g= g+ff
  g[0:n,0]= alpha*g[0:n,0]

  if iprint >= 4:
    print('Print g vector \n')
    print(g)
  return g

In [ ]:
def Citeration_additive_approx(C,f,alpha,n,p,K,iprint):

  # Perform additive Schwarz iteration
  # Compute contribution from each subdomain independently
  # Introduce random error in each subdomain computation
  
  # Store exact solutions from K iterations in a one-dimensional vector
  # Store approximate solutions from K iterations in a one-dimensional vector
  # Store residuals from K iterations in a one-dimensional vector

  veps= 1e-02;

  CC= np.zeros((n,n))
  ff= np.zeros((n,1))

  v= np.zeros((n,p))
  va= np.zeros((n,p))
  r= np.zeros((n,p))

  vglobal= np.zeros((n,1))
  vaglobal= np.zeros((n,1))
  rglobal_store= np.zeros((K*n,1))

  vexact_store= np.zeros((n*(K+1),1))
  vaglobal_store= np.zeros((n*(K+1),1))

  for k in range(K):
    if iprint >= 2:
        print('\nOuter iteration %3i \n',k)
  
    
    for i in range(p):
        # Compute the contributions from each subdomain
        CC[0:n,0:n]= C[i]#(i,1:n,1:n);
        ff[0:n,0:1]= f[i]#(i,1:n);
        v[0:n,i]= ff-CC@vglobal[0:n,0:1]
        va[0:n,i]= ff-CC@vaglobal[0:n,0:1] + veps*randn[n,1]

    # Perform update of global solution
    for i in range(p):
        vglobal[0:n,0:1]= vglobal[0:n,0:1] + alpha*v[0:n,i]
        vaglobal[0:n,0:1]= vaglobal[0:n,0:1] + alpha*va[0:n,i]

    # vglobal_store[(k+1)*n+1:(k+1)*n,0:1)= vglobal# [0:n,0:1]
    # vaglobal_store[(k+1)*n+1:(k+1)*n,0:1)= vaglobal #same
    #                                                         #need to fix indexing
    # diff= vglobal-vaglobal;
    # if iprint >= 2:
    #     print('Difference = %10.4e \n', norm(diff))
        
    # D= Dmatrix_additive(C,alpha,n,p,iprint);
    # g= gvector_additive(f,alpha,n,p,iprint);
    # rglobal= (g + D*vaglobal_store((k-1)*n+1:k*n,1)) - vaglobal(1:n,1);
    # rglobal_store((k-1)*n+1:k*n,1)= rglobal(1:n,1)
  
  return vglobal_store,vaglobal_store,rglobal_store



### For U iteration

In [ ]:
def Uhsolve_additive(U,h,n,K,iprint):
  # Perform additive Schwarz iterations
  # K iterations performed by single matrix solve
  # Store solutions from K iterations in a one-dimensional vector

  z= np.linalg.solve(U,h)
  if iprint > 0:
    for k in range(K):
        print(['z at iteration ',k,'\n'])
        zz[0:n,k]=z[(k-1)*n+1:k*n,1]
        print(zz[0:n,k])
  #return U or z #?

In [ ]:
def Umatrix_additive(D,n,K,iprint):

# Construct U matrix for additive Schwarz

  U= np.identity(K*n)

  for i in range(K-1):
    U[(i+1)*n:(i+2)*n, i*n:(i+1)*n]= -D

  if iprint >= 4:
    print('U matrix \n')
    print(U)
  return U


## Multiplicative Utilities

In [ ]:
def Dmatricies_multiplicative(C,n,p,iprint):
  D= np.zeros((p,n,n))
  for i in range(p):
    CC= C[i]
    DD= np.identity(n)-CC;
    D[i]= DD
  if iprint >= 4:
    print('Print DD matrix ')
    print(D)
  return D


In [ ]:
def fvector(B,y,n,p,iprint):
  f=np.zeros((p,n))
  BB=np.zeros((n,n))
  for i in range(p):
    BB=B[i]
    f[i]=BB@y[0:n,0]
  #need to add iprint
  #print('f',np.shape(f))
  #print(type(f))
  return f


In [ ]:
def Diteration_multiplicative(D,f,n,p,K,iprint):
  vstore=np.zeros((n*p*K,1))
  v=np.zeros((n,1))
  ff=np.zeros((1,n))
  #print('K=',K)
  for k in range(K):
    for i in range (p):
      DD=D[i]
      ff=f[i,range(n)]
      ff=ff[np.newaxis]
      v=DD@v+ff.T
      #print(np.shape(v))
      #print(np.shape(vstore))
      #print((k-1)*n*p+(i-1)*n)
      #print((k-1)*n*p+(i*n))
      vstore[k*n*p + i*n : k*n*p+(i+1)*n]=v[range(n)] #change i*n to i*(n+1)?
      if iprint>=4:
        print('Outer iteration k, Inner iteration i',k,i) #incomplete
        print(v.T)
    if iprint>=2:
      print('Outer iteration' , k)
      print(v.T)
  return vstore


### ST Iteration Multiplicative

In [ ]:
def Smatrix_multiplicative(D,n,p,iprint):
# Construct S matrix for multiplicative Schwarz

  S= np.zeros((n*p,n*p))
  for i in range(0,p):
    # print((i)*n,(i+1)*n, (i)*n,(i+1)*n)
    S[(i)*n:(i+1)*n, (i)*n:(i+1)*n] = np.identity(n)

  for i in range(1,p):
    DD=D[i]
    # print((i)*n+1,(i+1)*n, (i-1)*n+1,(i)*n)
    S[(i)*n:(i+1)*n, (i-1)*n:(i)*n] = -DD[0:n,0:n]

  if iprint >= 4:
    print('Print S matrix \n')
    print(S)
  return S


In [ ]:
def Tmatrix_multiplicative(D,n,p,iprint):

  # Construct T matrix for multiplicative Schwarz

  T= np.zeros((n*p,n*p))
  DD= D[0]
  T[0:n,(p-1)*n:p*n]= -DD

  if iprint >= 4:
    print('Print T matrix \n')
    print(T)
  return T


## Testing

In [ ]:
# test suite:
# show that each function functions as we say
# remove hardcoding
#   use limits
#     check set of rows for errors


separate into 4 files, multiple cells



test suite:

show that each function functions as we say

remove hardcoding

  use limits - K, n, p

  check set of rows for errors - verifydomains1

  ea subdomain must overlap another

  user can define subdomains -consecutive rows - definition of ne and nb

  rows can be in multiple>2 subdomains

  experiment 
  
  no holes - verifydomains1

  run experiment on disjoint set of subdomains - inconclusive





[14:23] Tavener,Simon
sub 1:   1 2 3 4 5

[14:23] Tavener,Simon
sub 2:  5 7 9 10

[14:24] Tavener,Simon
sub 4: 7 11 12 13 14 15

[14:25] Leautaud,Andre (EID)
sub 3: 6 8

[14:25] Leautaud,Andre (EID)
sub 5: 6, 16, 17, 18



In [ ]:
Additive_Schwarz(6,0.1,0,0)

[2.  2.5 3.  3.5 4. ]
[2.  2.5 3.  3.5 4. ]
(array([3, 3, 5, 4]), array([1, 2, 4, 7]), array([ 3,  4,  8, 10]), array([[ 1.,  2.,  3.,  0.,  0.],
       [ 2.,  3.,  4.,  0.,  0.],
       [ 4.,  5.,  6.,  7.,  8.],
       [ 7.,  8.,  9., 10.,  0.]]))
[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
4 10 10
top (10,)
(1, 10)
(10, 10)
top (10,)
(1, 10)
(10, 10)


In [ ]:
S=Multiplicative_Schwarz(6,0)
# print(S[0:10,0:10])
# print(S[10:20,0:10])
# print(S[0:10,10:20])
# print(S[10:20,10:20])
# print(S[20:40,0:10])


[2.  2.5 3.  3.5 4. ]
[2.  2.5 3.  3.5 4. ]
(array([3, 3, 5, 4]), array([1, 2, 4, 7]), array([ 3,  4,  8, 10]), array([[ 1.,  2.,  3.,  0.,  0.],
       [ 2.,  3.,  4.,  0.,  0.],
       [ 4.,  5.,  6.,  7.,  8.],
       [ 7.,  8.,  9., 10.,  0.]]))
[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
4 10 10
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0.

TypeError: ignored

In [ ]:
Multiplicative_Schwarz(10,0)

[2.  2.5 3.  3.5 4. ]
[2.  2.5 3.  3.5 4. ]
(array([3, 3, 5, 4]), array([1, 2, 4, 7]), array([ 3,  4,  8, 10]), array([[ 1.,  2.,  3.,  0.,  0.],
       [ 2.,  3.,  4.,  0.,  0.],
       [ 4.,  5.,  6.,  7.,  8.],
       [ 7.,  8.,  9., 10.,  0.]]))
[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
4 10 10
f (4, 10)
<class 'numpy.ndarray'>
K= 10
(10, 1)
(400, 

In [ ]:
n=np.zeros(3)
n[0:2]=np.linspace(1,2,2)
n

array([1., 2., 0.])

In [ ]:
np.linspace(1,3,3)

array([1., 2., 3.])

In [ ]:
Multiplicative_Schwarz(6)

[2.  2.5 3.  3.5 4. ]
[2.  2.5 3.  3.5 4. ]
(array([3, 3, 5, 4]), array([1, 2, 4, 7]), array([ 3,  4,  8, 10]), array([[ 1.,  2.,  3.,  0.,  0.],
       [ 2.,  3.,  4.,  0.,  0.],
       [ 4.,  5.,  6.,  7.,  8.],
       [ 7.,  8.,  9., 10.,  0.]]))
[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
4 10 10
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0.